# Step 1 : Token

In [30]:
import re
text="Hello, world. this, is the test."
result=re.split(r'([,.]|\s)',text)
print(result)


['Hello', ',', '', ' ', 'world', '.', '', ' ', 'this', ',', '', ' ', 'is', ' ', 'the', ' ', 'test', '.', '']


<div class="alert alert-block alert-info">
now we need to remove whitespace
    
but that is not mandatory suppose you are training python code or yml then whitespace having meaning 
item for item in result Iterates through each element of the list result.
if item.split() item.split() splits the string by whitespace.
If the string is empty or contains only spaces, item.split() returns an empty list:
 "" .split() → []
 " ".split() → []
 An empty list is treated as False in Python. So, the condition if item.split() becomes:
 True → keep the item
 False → discard the item
</div>

In [31]:

result=[item for item in result if item.split()]
print(result)

['Hello', ',', 'world', '.', 'this', ',', 'is', 'the', 'test', '.']


In [32]:

text="Hello, world. this, is-- the test?"
result=re.split(r'([,.:;?!"]|--|\s)',text)
print(result)
result=[item for item in result if item.split()]
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'this', ',', '', ' ', 'is', '--', '', ' ', 'the', ' ', 'test', '?', '']
['Hello', ',', 'world', '.', 'this', ',', 'is', '--', 'the', 'test', '?']


In [33]:
#in python open and read method present

with open("the-verdict.txt","r", encoding="utf-8")as f:
    raw_text = f.read()
    
print("total number of charecter", len(raw_text))
    
print(raw_text[:100])    



total number of charecter 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g


In [74]:


preprocess=re.split(r'([,.:;?!)("]|--|\s)',raw_text)
preprocess=[item for item in preprocess if item.split()]
token_size=len(preprocess)
print(" token size :", token_size)
print(preprocess[:30])



 token size : 4483
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


# Step 2 : creating token id


<div class="alert alert-block alert-info">
while we are genrating token id then first we need to create vocabulary. While creating vocabulary then all token will be in sorted in alphabetical order
</div>

In [99]:
all_world=sorted(set(preprocess))
vocablary_size=len(all_world)
print("vocablary size :",vocablary_size)

vocablary size : 1163


In [76]:

vocab={token:integer for integer,token in enumerate(all_world)}

#enumerate(all_world)  --->> gives (index, word)
#token: intrger.  --->> word : number
#vocab = {...} --->>creates dictionary

for i, (token, tokenid) in enumerate(vocab.items()):
    if i == 20:
        break
    print(token, tokenid)


! 0
" 1
' 2
'Are 3
'It's 4
'coming' 5
'done' 6
'subject 7
'technique' 8
'way 9
( 10
) 11
, 12
-- 13
. 14
: 15
; 16
? 17
A 18
Ah 19


In [58]:
#now here we can think we have token and token id mean encoder we are passing text then genrate token id
#but in decoder side we will pass tokenid then we need to get token text



<div class="alert alert-block alert-info">
    Now we are going to create a class called Tokenizer in python. In calss having encoder and decoder method
</div>

<div class="alert alert-block alert-info">
   Encoder method:  we are taking simple text then genrate token and assign tokenID
</div>


**Decoder method: it is reverse to encoder we are taking tokenId then convert it token after that simple text**


In [91]:
class SimpleTokenizerV1:
    def __init__(self,vocab):
        self.str_to_int=vocab
        self.int_to_str={tokenid:token for token,tokenid in vocab.items()}

    def encode(self,text):
        preprocess=re.split(r'([,.:;?!"]|--|\s)',text)
        preprocess=[item for item in preprocess if item.split()]
        ids=[self.str_to_int[s] for s in preprocess]
        return ids


    def decode(self, ids):
        # Join tokens back to text
        text = " ".join(self.int_to_str[i] for i in ids)
        # Remove space before punctuation
        text = re.sub(r'\s+([,.:;?!\"()\]])', r'\1', text)
        return text
    

In [107]:
tokenizer=SimpleTokenizerV1(vocab)
#Now we can test our encoder by passing some text from story which is loaded

text=""""It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""

ids=tokenizer.encode(text)
print(ids)

[1, 69, 1016, 634, 561, 778, 12, 1157, 628, 12, 1, 81, 14, 44, 880, 1139, 786, 825, 14]


In [109]:
tokenizer.decode(ids)

'" It\'s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

<div class="alert alert-block alert-info">
  if you pass some text which is not present in vocablary then it will give error that is is resion LLM is trained large amount of the data and real LLM like chat GPT it is using billion of data ans also added special context token for handling this king of error 
</div>


In [95]:
#text=""""Hello let us take tea"""
#ids=tokenizer.encode(text)
#print(ids)

# Special context token

In [116]:
all_token=sorted(set(preprocess))
all_token.extend(["<|endoftext|>","<|unk|>"])
vocablary_size=len(all_token)
print("vocablary size :",vocablary_size)
vocab1={token:integer for integer,token in enumerate(all_token)}


vocablary size : 1165


In [119]:
class SimpleTokenizerV2:
    def __init__(self,vocab):
        self.str_to_int=vocab
        self.int_to_str={tokenid:token for token,tokenid in vocab.items()}

    def encode(self,text):
        preprocess=re.split(r'([,.:;?!"]|--|\s)',text)
        preprocess=[item for item in preprocess if item.split()]
        preprocess=[item if item in self.str_to_int
                   else "<|unk|>" for item in preprocess
                   ]
        
        ids=[self.str_to_int[s] for s in preprocess]
        return ids


    def decode(self, ids):
        # Join tokens back to text
        text = " ".join(self.int_to_str[i] for i in ids)
        # Remove space before punctuation
        text = re.sub(r'\s+([,.:;?!\"()\]])', r'\1', text)
        return text





In [120]:
tokenizer1=SimpleTokenizerV2(vocab1)
text1=""""Hello let us take tea"""
ids1=tokenizer1.encode(text1)
print(ids1)

[1, 1164, 653, 1086, 1001, 1004]


In [121]:
tokenizer1.decode(ids1)

'" <|unk|> let us take tea'

<div class="alert alert-warning">
  Let test <|endoftext|>
</div>

In [124]:
text8=""""Hello let us take tea"""
text10="""And his tone told me in a flash that he never thought of anything else."""
newtext=" <|endoftext|> ".join((text8,text10))
print(newtext)

"Hello let us take tea <|endoftext|> And his tone told me in a flash that he never thought of anything else.


In [125]:
tokenizer1.encode(newtext)

[1,
 1164,
 653,
 1086,
 1001,
 1004,
 1163,
 21,
 579,
 1048,
 1047,
 695,
 600,
 147,
 475,
 1015,
 561,
 739,
 1033,
 754,
 193,
 415,
 14]

In [127]:
newids=tokenizer1.encode(newtext)
print(newids)

[1, 1164, 653, 1086, 1001, 1004, 1163, 21, 579, 1048, 1047, 695, 600, 147, 475, 1015, 561, 739, 1033, 754, 193, 415, 14]


In [128]:
tokenizer1.decode(newids)

'" <|unk|> let us take tea <|endoftext|> And his tone told me in a flash that he never thought of anything else.'